# DWH ETL

Importeer dependencies:

In [ ]:
import sqlite3
import pandas as pd
import pyodbc
import re

Connectie met SDM-database (import) en DWH-database (export) in SSMS:

In [ ]:
DB_SDM = {"servername": r"143.177.235.181", "database": "SDM_DEDS_P1", "username": "sa", "password": "Str0ngP@ssword"}
DB_DWH = {"servername": r"143.177.235.181", "database": "Datawarehouse_DEDS_P2", "username": "sa", "password": "Str0ngP@ssword"}
import_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_SDM['servername']};"
    f"DATABASE={DB_SDM['database']};"
    f"UID={DB_SDM['username']};"
    f"PWD={DB_SDM['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

export_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_DWH['servername']};"
    f"DATABASE={DB_DWH['database']};"
    f"UID={DB_DWH['username']};"
    f"PWD={DB_DWH['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

DataFrames maken voor SDM-tables (Extract):

In [ ]:
def create_dataframes_sql(connection):
    dictionary : dict = {}
    query : str = ""
    key : str = ""

    query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
    key = "TABLE_NAME"

    table_names = pd.read_sql(query, connection)

    for table in table_names[key].tolist():
        try:
            dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
        except Exception as e:
            print(f"Error reading {table}: {e}")
            continue

    return dictionary

sdm = create_dataframes_sql(import_conn)

TRANSFORMeren naar DWH-model: